In [1]:
import os
import numpy as np
import tensorflow as tf
from keras import models, layers, regularizers

from loop import TrainingLoop
from batch_selection import windowed_batch_selector, sorting_batch_selector

In [2]:
# Loading the dataset from the files saved in the preprocessing notebook.
path = 'data/music_genres'
prefix = 'music_genres_'
X_train = np.load(os.path.join(path, prefix+'train_vectors.npy'))
y_train = np.load(os.path.join(path, prefix+'train_labels.npy'))
X_test  = np.load(os.path.join(path, prefix+'test_vectors.npy'))
y_test  = np.load(os.path.join(path, prefix+'test_labels.npy'))

In [3]:
def build_model():
    # Setting up the model.
    model = models.Sequential()
    model.add(layers.Dense(250, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(100, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(50, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(layers.Dense(y_train.shape[1], activation='softmax'))
    return model

In [4]:
def train(model, X_train, y_train, batch_selection, epochs):
    log_dir = {windowed_batch_selector: 'windowed', sorting_batch_selector: 'sorting', None: 'original'}
    # Put the model in our custom training loop.
    TrainingLoop(
        model,
        X_train,
        y_train,
        validation_split = 0.1,
        batch_size = 12,
        optimizer = tf.keras.optimizers.Adam(),
        loss_function = tf.keras.losses.CategoricalCrossentropy(),
        train_metrics = tf.keras.metrics.CategoricalAccuracy(),
        val_metrics = tf.keras.metrics.CategoricalAccuracy(),
        batch_selection = batch_selection,
        log_file = os.path.join('logs', log_dir[batch_selection], 'music_genres.csv')
    ).train(epochs)  # Training model.

In [5]:
# Set random seed so the comparison of different solutions won't be affected by it.
tf.random.set_seed(42)
np.random.seed(42)

# Train model with default batch selection.
model = build_model()
train(model, X_train, y_train, batch_selection=None, epochs=50)

# We still have to compile the model for the test evaluation.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# After compiling we can run the evaluation.
model.evaluate(X_test, y_test)

Epoch 1/50	Loss: 1.8535	Metrics: 0.2055: 	Validation metrics: 0.375: 	100% | 60/60 [00:02<00:00, 24.67it/s]
Epoch 2/50	Loss: 1.5358	Metrics: 0.3638: 	Validation metrics: 0.4444: 	100% | 60/60 [00:00<00:00, 69.28it/s]
Epoch 3/50	Loss: 1.4155	Metrics: 0.4583: 	Validation metrics: 0.5138: 	100% | 60/60 [00:00<00:00, 77.62it/s]
Epoch 4/50	Loss: 1.6259	Metrics: 0.4819: 	Validation metrics: 0.4722: 	100% | 60/60 [00:00<00:00, 83.80it/s]
Epoch 5/50	Loss: 1.3714	Metrics: 0.5430: 	Validation metrics: 0.5138: 	100% | 60/60 [00:01<00:00, 49.88it/s]
Epoch 6/50	Loss: 1.4894	Metrics: 0.5722: 	Validation metrics: 0.5277: 	100% | 60/60 [00:00<00:00, 70.01it/s]
Epoch 7/50	Loss: 1.5168	Metrics: 0.5638: 	Validation metrics: 0.5277: 	100% | 60/60 [00:00<00:00, 82.08it/s]
Epoch 8/50	Loss: 1.6092	Metrics: 0.6013: 	Validation metrics: 0.5416: 	100% | 60/60 [00:00<00:00, 87.34it/s]
Epoch 9/50	Loss: 1.4755	Metrics: 0.6041: 	Validation metrics: 0.5833: 	100% | 60/60 [00:00<00:00, 86.45it/s]
Epoch 10/50	Loss: 1.

[5.233212947845459, 0.6850000023841858]

In [6]:
tf.random.set_seed(42)
np.random.seed(42)

# Train model with windowed batch selection algorithm.
model = build_model()
train(model, X_train, y_train, batch_selection=windowed_batch_selector, epochs=50)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.evaluate(X_test, y_test)

Epoch 1/50	Loss: 2.0496	Metrics: 0.1958: 	Validation metrics: 0.4305: 	100% | 60/60 [00:06<00:00, 10.00it/s]
Epoch 2/50	Loss: 1.5194	Metrics: 0.2874: 	Validation metrics: 0.4166: 	100% | 60/60 [00:01<00:00, 42.64it/s]
Epoch 3/50	Loss: 1.4034	Metrics: 0.3597: 	Validation metrics: 0.4583: 	100% | 60/60 [00:01<00:00, 44.38it/s]
Epoch 4/50	Loss: 1.6548	Metrics: 0.4486: 	Validation metrics: 0.4722: 	100% | 60/60 [00:01<00:00, 43.76it/s]
Epoch 5/50	Loss: 1.3414	Metrics: 0.4430: 	Validation metrics: 0.4722: 	100% | 60/60 [00:01<00:00, 42.95it/s]
Epoch 6/50	Loss: 1.3222	Metrics: 0.4986: 	Validation metrics: 0.5555: 	100% | 60/60 [00:01<00:00, 44.35it/s]
Epoch 7/50	Loss: 1.5119	Metrics: 0.5319: 	Validation metrics: 0.5555: 	100% | 60/60 [00:01<00:00, 44.15it/s]
Epoch 8/50	Loss: 1.6802	Metrics: 0.5402: 	Validation metrics: 0.5277: 	100% | 60/60 [00:01<00:00, 43.04it/s]
Epoch 9/50	Loss: 1.2591	Metrics: 0.5680: 	Validation metrics: 0.5277: 	100% | 60/60 [00:01<00:00, 40.00it/s]
Epoch 10/50	Loss: 1

[5.26779317855835, 0.6600000262260437]

In [7]:
tf.random.set_seed(42)
np.random.seed(42)

# Train model with sorting batch selection algorithm.
model = build_model()
train(model, X_train, y_train, batch_selection=sorting_batch_selector, epochs=50)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.evaluate(X_test, y_test)

Epoch 1/50	Loss: 1.6581	Metrics: 0.2722: 	Validation metrics: 0.4166: 	100% | 60/60 [00:04<00:00, 13.98it/s]
Epoch 2/50	Loss: 1.0616	Metrics: 0.3958: 	Validation metrics: 0.4027: 	100% | 60/60 [00:00<00:00, 61.35it/s]
Epoch 3/50	Loss: 0.8526	Metrics: 0.4402: 	Validation metrics: 0.4583: 	100% | 60/60 [00:00<00:00, 60.24it/s]
Epoch 4/50	Loss: 0.7208	Metrics: 0.4763: 	Validation metrics: 0.5: 	100% | 60/60 [00:01<00:00, 56.76it/s]
Epoch 5/50	Loss: 0.7095	Metrics: 0.5069: 	Validation metrics: 0.4861: 	100% | 60/60 [00:00<00:00, 62.18it/s]
Epoch 6/50	Loss: 0.7292	Metrics: 0.5430: 	Validation metrics: 0.5138: 	100% | 60/60 [00:01<00:00, 53.19it/s]
Epoch 7/50	Loss: 0.4974	Metrics: 0.5847: 	Validation metrics: 0.5555: 	100% | 60/60 [00:01<00:00, 56.76it/s]
Epoch 8/50	Loss: 0.5229	Metrics: 0.5847: 	Validation metrics: 0.5416: 	100% | 60/60 [00:01<00:00, 47.73it/s]
Epoch 9/50	Loss: 0.5416	Metrics: 0.6152: 	Validation metrics: 0.5694: 	100% | 60/60 [00:01<00:00, 57.47it/s]
Epoch 10/50	Loss: 0.47

[5.26262092590332, 0.6650000214576721]